# Data preperation 

The datasets from the IMDB database are normalised to atleast third degree, hence this notebook will take the the necassary fields of information and put it together for quick data retreival. This is essential as we want our system to be as quick as possible.

In [1]:
##importing libraries
import pandas as pd #for data manipulation
import unidecode #to replace accents with english letters

## Creating dataframe with Movie/Show information

### 1. Starting with main title file with language information

In [2]:
#reading the title files
df = pd.read_csv('datasets from IMDB/title_akas.tsv',sep='\t')

/Users/shashanksharma/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [4]:
df.language.unique()

array(['\\N', 'ja', 'sv', 'en', 'tr', 'es', 'sr', 'cs', 'fa', 'fr', 'bg',
       'ca', 'nl', 'qbn', 'pt', 'ru', 'uk', 'qbp', 'ar', 'cmn', 'rn',
       'bs', 'de', 'hi', 'yi', 'qbo', 'ka', 'hr', 'sl', 'he', 'tg', 'sk',
       'kk', 'da', 'el', 'fi', 'it', 'gsw', 'yue', 'az', 'ms', 'pl', 'mr',
       'uz', 'gl', 'th', 'ta', 'eu', 'be', 'af', 'la', 'hy', 'ur', 'bn',
       'te', 'lt', 'mk', 'et', 'lv', 'gd', 'tl', 'cy', 'id', 'qal', 'gu',
       'ml', 'ro', 'hu', 'pa', 'kn', 'wo', 'no', 'is', 'sq', 'zh', 'ps',
       'nqo', 'sd', 'ga', 'xh', 'mi', 'zu', 'ku', 'rm', 'prs', 'ky', 'vi',
       'fro', 'ko', 'haw', 'mn', 'lo', 'my', 'am', 'qac', 'ne', 'myv',
       'br', 'iu', 'st', 'tn', 'cr'], dtype=object)

As observed above there are movies/shows from multiple languages in IMDB database. But, this project is only focussing on English and Hindi movies. Hence, rows of information for all other languages are removed

In [5]:
#Keeping titles with english or hindi titles and removing every other row of information
selected_languages=['en','hi']
df = df[df.language.isin(selected_languages)]

Also, columns like ordering,region, type, attributes and isOriginalTitle are of no use to the recommendation system. Hence, these columns are removed below.

In [6]:
df = df.drop(labels=['region','types','attributes','isOriginalTitle','ordering'],axis=1)

In [7]:
df.head()

,titleId,title,language
95,tt0000012,The Arrival of a Train,en
97,tt0000012,The Arrival of a Train at La Ciotat,en
107,tt0000012,The Arrival of a Train,en
157,tt0000016,Boat Leaving the Port,en
239,tt0000029,Baby's Meal,en


### 2. Adding genre, year and title type(movie or show) information

In [8]:
#reading the file with basic title information 
basic = pd.read_csv('datasets from IMDB/title_basics.tsv',sep='\t')

/Users/shashanksharma/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
basic.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


Next, the basic data is joined with the title information. 

In [10]:
#joining the required columns from the the basic tile dataframe
basic = basic.set_index('tconst')
df_with_basic = pd.merge(df,basic[['titleType','isAdult','startYear','genres']],left_on='titleId',right_on='tconst',how='left')

In [11]:
df_with_basic.head()

,titleId,title,language,titleType,isAdult,startYear,genres
0,tt0000012,The Arrival of a Train,en,short,0.0,1896,"Action,Documentary,Short"
1,tt0000012,The Arrival of a Train at La Ciotat,en,short,0.0,1896,"Action,Documentary,Short"
2,tt0000012,The Arrival of a Train,en,short,0.0,1896,"Action,Documentary,Short"
3,tt0000016,Boat Leaving the Port,en,short,0.0,1895,"Documentary,Short"
4,tt0000029,Baby's Meal,en,short,0.0,1895,"Documentary,Short"


Checking the types of titles in the data

In [12]:
df_with_basic.titleType.unique()

array(['short', 'movie', 'tvSeries', 'tvMovie', 'tvMiniSeries',
       'tvEpisode', 'tvShort', 'video', 'videoGame', 'tvSpecial', nan],
      dtype=object)

Only movies, tvSeries and TVMiniSeries are relavent to this project. Hence, every row of information with other title type is removed.

In [13]:
to_keep = ['movie','tvSeries','tvMiniSeries']
df_with_basic = df_with_basic[df_with_basic['titleType'].isin(to_keep)]

Checking the genres column for values.

In [14]:
df_with_basic.genres.unique()

array(['\\N', 'Drama', 'Drama,Romance', ...,
       'Musical,Reality-TV,Talk-Show', 'Comedy,Short,Talk-Show',
       'Music,Musical,Reality-TV'], dtype=object)

In [15]:
df_with_basic[df_with_basic.genres == '\\N']

,titleId,title,language,titleType,isAdult,startYear,genres
68,tt0000838,The Cultivation of the Cacao Tree,en,movie,0.0,1909,\N
78,tt0001051,Magical Dream,en,movie,0.0,1909,\N
80,tt0001122,The Red Inn,en,movie,0.0,1910,\N
112,tt0002329,Today and Tomorrow,en,movie,0.0,1912,\N
126,tt0002801,The Black Diamond,en,movie,0.0,1913,\N
...,...,...,...,...,...,...,...
2651813,tt9908394,Sex Documentary: Meaty,en,movie,0.0,1981,\N
2651969,tt9909276,Documentary Porn: Compulsive Rapist,en,movie,0.0,1981,\N
2651992,tt9909736,Porno Documentary: Housewife's Prostitution Team,en,movie,0.0,1981,\N
2651993,tt9909744,Please Seduce Me with Dirty Words,en,movie,0.0,1981,\N


Movie without genres ar of no use to this project. Hence they are removed.

In [16]:
#removing rows with \N as a genre
df_with_basic = df_with_basic[~(df_with_basic.genres =='\\N')]

Some english movie are mentioned twice in the dataste both in Hindi and English languages. This could have happened as english movies are often dubbed in hindi to reach to a wider audience. One such example is shown below

In [17]:
df_with_basic[df_with_basic.title == 'Coffee & Kareem']

,titleId,title,language,titleType,isAdult,startYear,genres
2649423,tt9898858,Coffee & Kareem,en,movie,0.0,2020,"Action,Comedy"
2649424,tt9898858,Coffee & Kareem,hi,movie,0.0,2020,"Action,Comedy"


In such cases, The row with langauge as hindi are removed by sorting the dataframe be language column so that 'en'comes first and the drop duplicates is used to drop the second occurance of the title.

In [18]:
#Sorting the dataframe by langauge column
df_with_basic.sort_values(by=['language'],inplace=True)
#dropping second occurance of each title
df_with_basic.drop_duplicates(subset='title', keep='first',inplace=True)

Checking again.

In [19]:
#checking again to confirm that Coffee & Kareem only appears in Enlglish language
df_with_basic[df_with_basic.title == 'Coffee & Kareem']

,titleId,title,language,titleType,isAdult,startYear,genres
2649423,tt9898858,Coffee & Kareem,en,movie,0.0,2020,"Action,Comedy"


### 3.  Adding ratings information

In [20]:
ratings = pd.read_csv('datasets from IMDB/title_rating.tsv',sep='\t')

In [21]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1608
1,tt0000002,6.0,197
2,tt0000003,6.5,1286
3,tt0000004,6.1,121
4,tt0000005,6.1,2051


Joining the ratings information with the dataframe created above.

In [22]:
ratings = ratings.set_index('tconst')
df_with_ratings = pd.merge(df_with_basic,ratings,left_on='titleId',right_on='tconst',how='left')

In [23]:
df_with_ratings.head()

,titleId,title,language,titleType,isAdult,startYear,genres,averageRating,numVotes
0,tt0000941,Love Crazy,en,movie,0.0,1909,Drama,4.2,13.0
1,tt1919184,Vares: Tango of Darkness,en,movie,0.0,2012,"Crime,Drama,Thriller",5.3,607.0
2,tt1919137,The Last Supper,en,movie,0.0,2012,"Action,History",5.7,693.0
3,tt1919137,King's Feast,en,movie,0.0,2012,"Action,History",5.7,693.0
4,tt1919137,King's War,en,movie,0.0,2012,"Action,History",5.7,693.0


In [24]:
df_with_ratings.shape

(104319, 9)

In [25]:
df_with_ratings['title'] = df_with_ratings['title'].apply(lambda x: unidecode.unidecode(x))

In [26]:
df_with_ratings.to_csv("title_information.csv")

## Creating dataframe with cast information for each title

### 1. Creating dataframe with actor, actress and director infromation

Reading principal actor info

In [27]:
cast_info = pd.read_csv('datasets from IMDB/title_principal.tsv',sep='\t')

In [28]:
cast_info.head(20)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
5,tt0000003,1,nm0721526,director,\N,\N
6,tt0000003,2,nm5442194,producer,producer,\N
7,tt0000003,3,nm1335271,composer,\N,\N
8,tt0000003,4,nm5442200,editor,\N,\N
9,tt0000004,1,nm0721526,director,\N,\N


Removin titles in cast datafrme which are not present in the datafrme created above.

In [29]:
cast_info = cast_info[cast_info.tconst.isin(df_with_basic.titleId)]

Keeping the inforamtion of actor, actress and director and removing everything else

In [30]:
cast_info = cast_info[(cast_info.category == 'actor') | (cast_info.category == 'actress') | (cast_info.category == 'director')]

Removing the columns which are not required for the project. 

In [31]:
cast_info.drop(labels=['ordering','job','characters'], axis=1,inplace=True)

In [32]:
cast_info.head()


,tconst,nconst,category
3159,tt0000941,nm0034453,actor
3160,tt0000941,nm0140054,actor
3161,tt0000941,nm0243918,actor
3162,tt0000941,nm0294022,actress
3163,tt0000941,nm0063413,director


### 2. Replacing the codes for cast with their names

In [33]:
name_info = pd.read_csv('datasets from IMDB/name_basics.tsv',sep='\t')

In [34]:
name_info.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0043044,tt0072308,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0071877,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0059956,tt0049189,tt0054452,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0080455,tt0072562,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0050976,tt0050986,tt0060827"


In [35]:
#replacing the codes in the cast dataframe from names
cast_info['nconst'] = cast_info['nconst'].map(name_info.set_index('nconst')['primaryName'])

In [36]:
#renaming names column
cast_info.rename(columns = {'nconst':'name'}, inplace = True)

In [37]:
cast_info.head()

,tconst,name,category
3159,tt0000941,José Argelagués,actor
3160,tt0000941,Joaquín Carrasco,actor
3161,tt0000941,José Durany,actor
3162,tt0000941,Elvira Fremont,actress
3163,tt0000941,Ricardo de Baños,director


Changing the values in name column to string and then replacing accents with unaccanted letters.

In [38]:
cast_info['name'] = cast_info['name'].apply(str)
cast_info['name'] = cast_info['name'].apply(lambda x: unidecode.unidecode(x))

Writing to csv file

In [39]:
cast_info = cast_info.reset_index()
cast_info.to_csv('cast_infromation.csv',encoding="UTF-8")